In [1]:
s = 'TAGG'

# Build mismatch tree

In [2]:
def mismatchTree(s, k):
    """
    Builds mismatch tree for a string s up to k mismatches
    """
    if len(s)==1 and k==0:
        return [s]
    elif len(s)==1 and k>0:
        return ['T', 'A', 'C', 'G']
    elif len(s) > 1 and k==0:
        return {s[0]:mismatchTree(s[1:], 0)}
    
    else:
        d = {}
        for nucleotide in ['T', 'A', 'C', 'G']:
            if nucleotide == s[0]: # i.e. no mismatch at this node
                d[nucleotide] = mismatchTree(s[1:], k)
            else: # insert mismatch
                d[nucleotide] = mismatchTree(s[1:], k-1)
    return d

In [3]:
mismatch_tree_example = mismatchTree(s, 1)

In [4]:
import json

In [5]:
print(json.dumps(mismatch_tree_example, indent=4, sort_keys=True))

{
    "A": {
        "A": {
            "G": [
                "G"
            ]
        }
    },
    "C": {
        "A": {
            "G": [
                "G"
            ]
        }
    },
    "G": {
        "A": {
            "G": [
                "G"
            ]
        }
    },
    "T": {
        "A": {
            "A": [
                "G"
            ],
            "C": [
                "G"
            ],
            "G": [
                "T",
                "A",
                "C",
                "G"
            ],
            "T": [
                "G"
            ]
        },
        "C": {
            "G": [
                "G"
            ]
        },
        "G": {
            "G": [
                "G"
            ]
        },
        "T": {
            "G": [
                "G"
            ]
        }
    }
}


In [6]:
'TACG'

'TACG'

In [7]:
def isInMismatchTree(mismatch_tree, s):
    current_sub_tree = mismatch_tree.copy()
    
    for letter in s:
    
        if isinstance(current_sub_tree, list):
            if letter in current_sub_tree:
                return True
            else:
                return False
        
        else:
            if letter in current_sub_tree.keys():
                current_sub_tree = current_sub_tree[letter]
            else:
                return False

In [8]:
isInMismatchTree(mismatchTree('TACG', 1), 'TACG')

True

In [9]:
isInMismatchTree(mismatchTree('TACC', 1), 'TAGG')

False

# Import from py file

In [11]:
def inMismatchTree(mismatch_tree, keys):
    return [key for key in keys if isInMismatchTree(mismatch_tree, key)]

In [15]:
inMismatchTree(mismatchTree('TACC', 1), ['TAGG', 'TGGG', 'TACG', 'TGCC'])

['TACG', 'TGCC']

In [259]:
mismatchTree('TACC', 1)

{'A': {'A': {'C': 'C'}},
 'C': {'A': {'C': 'C'}},
 'G': {'A': {'C': 'C'}},
 'T': {'A': {'A': 'C', 'C': 'C', 'G': 'C', 'T': 'C'},
  'C': {'C': 'C'},
  'G': {'C': 'C'},
  'T': {'C': 'C'}}}